In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
train = pd.read_csv('/kaggle/input/titanic/train.csv')
test = pd.read_csv('/kaggle/input/titanic/test.csv')
gs = pd.read_csv('/kaggle/input/titanic/gender_submission.csv')

# Step 1: **Analyzing and Cleaning Dataset**

In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [4]:
train.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
PassengerId,891.0,NaN,NaN,NaN,446.0,257.353842,1.0,223.5,446.0,668.5,891.0
Survived,891.0,NaN,NaN,NaN,0.383838,0.486592,0.0,0.0,0.0,1.0,1.0
Pclass,891.0,NaN,NaN,NaN,2.308642,0.836071,1.0,2.0,3.0,3.0,3.0
Name,891,891,"Braund, Mr. Owen Harris",1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Sex,891,2,male,577,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Age,714.0,NaN,NaN,NaN,29.699118,14.526497,0.42,20.125,28.0,38.0,80.0
SibSp,891.0,NaN,NaN,NaN,0.523008,1.102743,0.0,0.0,0.0,1.0,8.0
Parch,891.0,NaN,NaN,NaN,0.381594,0.806057,0.0,0.0,0.0,0.0,6.0
Ticket,891,681,347082,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Fare,891.0,NaN,NaN,NaN,32.204208,49.693429,0.0,7.9104,14.4542,31.0,512.3292


In [5]:
train['Survived'].value_counts(normalize=True)

0    0.616162
1    0.383838
Name: Survived, dtype: float64

In [6]:
train.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [7]:
# import seaborn as sns
# sns.pairplot(df, hue="Survived", palette='pastel')

In [8]:
object_train = [i for i in train.columns if train[i].dtypes == 'object']
object_train

['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']

In [9]:
x = train["Age"].median()
train["Age"].fillna(x, inplace = True)

y = train["Fare"].median()
train["Fare"].fillna(y, inplace = True)

train["Embarked"].fillna(method="ffill", inplace = True)

In [10]:
a = test["Age"].median()
test["Age"].fillna(a, inplace = True)

b = test["Fare"].median()
test["Fare"].fillna(b, inplace = True)

test["Embarked"].fillna(method="ffill", inplace = True)

In [11]:
train.isna().mean()

PassengerId    0.000000
Survived       0.000000
Pclass         0.000000
Name           0.000000
Sex            0.000000
Age            0.000000
SibSp          0.000000
Parch          0.000000
Ticket         0.000000
Fare           0.000000
Cabin          0.771044
Embarked       0.000000
dtype: float64

In [12]:
train = train.drop(columns=['Cabin', 'Name', 'Ticket'], axis=1)
test = test.drop(columns=['Cabin', 'Name', 'Ticket'], axis=1)
train

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,male,22.0,1,0,7.2500,S
1,2,1,1,female,38.0,1,0,71.2833,C
2,3,1,3,female,26.0,0,0,7.9250,S
3,4,1,1,female,35.0,1,0,53.1000,S
4,5,0,3,male,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...,...,...
886,887,0,2,male,27.0,0,0,13.0000,S
887,888,1,1,female,19.0,0,0,30.0000,S
888,889,0,3,female,28.0,1,2,23.4500,S
889,890,1,1,male,26.0,0,0,30.0000,C


In [13]:
from sklearn.preprocessing import OneHotEncoder

In [14]:
enc = OneHotEncoder(sparse=False)
columns_to_one_hot = ['Sex','Embarked']

encoded_array_train = enc.fit_transform(train.loc[:,columns_to_one_hot])
encoded_array_test = enc.fit_transform(test.loc[:,columns_to_one_hot])

train_encoded = pd.DataFrame(encoded_array_train,columns=enc.get_feature_names_out() )
test_encoded = pd.DataFrame(encoded_array_test,columns=enc.get_feature_names_out() )

train_encoded = pd.concat([train,train_encoded],axis=1)
test_encoded = pd.concat([test,test_encoded],axis=1)

train_encoded.drop(labels= columns_to_one_hot,axis=1,inplace=True)
test_encoded.drop(labels= columns_to_one_hot,axis=1,inplace=True)


In [15]:
test_encoded

,PassengerId,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,892,3,34.5,0,0,7.8292,0.0,1.0,0.0,1.0,0.0
1,893,3,47.0,1,0,7.0000,1.0,0.0,0.0,0.0,1.0
2,894,2,62.0,0,0,9.6875,0.0,1.0,0.0,1.0,0.0
3,895,3,27.0,0,0,8.6625,0.0,1.0,0.0,0.0,1.0
4,896,3,22.0,1,1,12.2875,1.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,27.0,0,0,8.0500,0.0,1.0,0.0,0.0,1.0
414,1306,1,39.0,0,0,108.9000,1.0,0.0,1.0,0.0,0.0
415,1307,3,38.5,0,0,7.2500,0.0,1.0,0.0,0.0,1.0
416,1308,3,27.0,0,0,8.0500,0.0,1.0,0.0,0.0,1.0


In [16]:
X = train_encoded.drop(columns=['Survived']).copy()
y = train_encoded['Survived'].copy()

In [17]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [18]:
X_mm = pd.DataFrame(scaler.fit_transform(X), columns = X.columns)

In [19]:
X_mm.head()

,PassengerId,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,0.000000,1.0,0.271174,0.125,0.0,0.014151,0.0,1.0,0.0,0.0,1.0
1,0.001124,0.0,0.472229,0.125,0.0,0.139136,1.0,0.0,1.0,0.0,0.0
2,0.002247,1.0,0.321438,0.000,0.0,0.015469,1.0,0.0,0.0,0.0,1.0
3,0.003371,0.0,0.434531,0.125,0.0,0.103644,1.0,0.0,0.0,0.0,1.0
4,0.004494,1.0,0.434531,0.000,0.0,0.015713,0.0,1.0,0.0,0.0,1.0


# Step 3: **Logistic Regression**

In [20]:
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression(solver = 'liblinear', penalty = 'l2', C=2)

In [21]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_mm, y, test_size=0.33, random_state=42)

In [22]:
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings('ignore')

parameters = {
    'penalty' : ['l1','l2'], 
    'C'       : range(1,15),
    'solver'  : ['liblinear', 'newton-cg', 'lbfgs', 'sag', 'saga'],
}

In [23]:
clf = GridSearchCV(LR,               
                   param_grid = parameters,  
                   scoring='roc_auc',  
                   cv=10)

In [24]:
clf.fit(X_train,y_train)

GridSearchCV(cv=10, estimator=LogisticRegression(C=2, solver='liblinear'),
             param_grid={'C': range(1, 15), 'penalty': ['l1', 'l2'],
                         'solver': ['liblinear', 'newton-cg', 'lbfgs', 'sag',
                                    'saga']},
             scoring='roc_auc')

In [25]:
print("Tuned Hyperparameters :", clf.best_params_)
print("Accuracy :",clf.best_score_)

Tuned Hyperparameters : {'C': 2, 'penalty': 'l2', 'solver': 'liblinear'}
Accuracy : 0.8284471126004307


In [26]:
LR.fit(X_train, y_train)

LogisticRegression(C=2, solver='liblinear')

In [27]:
predicted = LR.predict(X_test)

In [28]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score

In [29]:
scores = {'accuracy': accuracy_score(y_test, predicted, normalize=True),
         'f1': f1_score(y_test, predicted),
         'roc': roc_auc_score(y_test, predicted)}
scores

{'accuracy': 0.8067796610169492,
 'f1': 0.748898678414097,
 'roc': 0.7913095238095239}

In [30]:
test_mm = pd.DataFrame(scaler.fit_transform(test_encoded), columns = test_encoded.columns)

In [31]:
predict_sub = LR.predict(test_mm)

In [32]:
predict_sub

array([0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0,
       1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [33]:
submition = pd.DataFrame({'PassengerId':test['PassengerId'], 'Survived': predict_sub})
submition.to_csv(index=False)

'PassengerId,Survived\n892,0\n893,0\n894,0\n895,0\n896,1\n897,0\n898,1\n899,0\n900,1\n901,0\n902,0\n903,0\n904,1\n905,0\n906,1\n907,1\n908,0\n909,0\n910,1\n911,1\n912,0\n913,0\n914,1\n915,1\n916,1\n917,0\n918,1\n919,0\n920,0\n921,0\n922,0\n923,0\n924,0\n925,0\n926,0\n927,0\n928,1\n929,1\n930,0\n931,0\n932,0\n933,0\n934,0\n935,1\n936,1\n937,0\n938,0\n939,0\n940,1\n941,0\n942,0\n943,0\n944,1\n945,1\n946,0\n947,0\n948,0\n949,0\n950,0\n951,1\n952,0\n953,0\n954,0\n955,1\n956,1\n957,1\n958,1\n959,0\n960,1\n961,1\n962,1\n963,0\n964,1\n965,1\n966,1\n967,1\n968,0\n969,1\n970,0\n971,1\n972,0\n973,0\n974,0\n975,0\n976,0\n977,0\n978,1\n979,1\n980,1\n981,0\n982,1\n983,0\n984,1\n985,0\n986,1\n987,0\n988,1\n989,0\n990,1\n991,0\n992,1\n993,0\n994,0\n995,0\n996,1\n997,0\n998,0\n999,0\n1000,0\n1001,0\n1002,0\n1003,1\n1004,1\n1005,1\n1006,1\n1007,0\n1008,0\n1009,1\n1010,1\n1011,1\n1012,1\n1013,0\n1014,1\n1015,0\n1016,0\n1017,1\n1018,0\n1019,1\n1020,0\n1021,0\n1022,0\n1023,0\n1024,1\n1025,0\n1026,0\n1027,

In [34]:
import base64
from IPython.display import HTML

In [35]:
def create_download_link(df, title = "Download CSV file", filename = "sub.csv"):  
    csv = df.to_csv()
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

In [36]:
create_download_link(submition)